In [1]:
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 144

In [2]:
import grader

# Mapreduce

## Introduciton

We are going to be running mapreduce jobs on the wikipedia dataset.  The dataset is available (pre-chunked) on [s3](s3://dataincubator-course/mrdata/simple/).

For development, you can even use a single chunk (eg. part-00026.xml.bz2). That is small enough that mrjob can process the chunk in a few seconds. Your development cycle should be:

1.  Get your job to work locally on one chunk.  This will greatly speed up your
development.  To run on local:
```bash
python job_file.py -r local data/wikipedia/simple/part-00026.xml.bz2 > /tmp/output.txt
```
    
2.  Get your job to work on the full dataset on GCP (Google Cloud Platform).  This will greatly speed up your production.  To run on GCP ([details](https://pythonhosted.org/mrjob/guides/dataproc-quickstart.html)):
```bash
python job_file.py -r dataproc data/wikipedia/simple/part-00026.xml.bz2 \
    --output-dir=gs://my-bucket/output/ \
    --no-output 
```

    Not that you can also pass an entire local directory of data (eg. `data/simple/`) as the input.

### Note on Memory
There's a large difference between developing locally on one chunk and running your job on the entire dataset.  While you can get away with sloppy memory use locally, you really need to keep memory usage down if you hope to be able to complete the miniproject.  Remember, memory needs to be $O(1)$, not $O(n)$ in input.

### Multiple Mapreduces
You can combine multiple steps by overriding the [steps method](https://pythonhosted.org/mrjob/guides/writing-mrjobs.html#multi-step-jobs).  Usually your mapreduce might look like this
```python
from mrjob.job import MRJob

class SingleMRJob(MRJob):
    def mapper(self, key, value):
        pass

    def reducer(self, key, values):
        pass
```

`MRJob` automatically uses the `mapper` and `reducer` methods.  To specify multiple steps, you need to override the `steps` method:

```python
from mrjob.job import MRJob
from mrjob.step import MRStep

class MultipleMRJob(MRJob):
    def mapper1(self, key, value):
        pass

    def reducer1(self, key, values):
        pass
        
    def mapper2(self, key, value):
        pass

    def reducer2(self, key, values):
        pass
        
    def steps(self):
        return [
            MRStep(mapper=self.mapper1, reducer=self.reducer1),
            MRStep(mapper=self.mapper2, reducer=self.reducer2),
        ]
```

As a matter of good style, we recommend that you actually write each individual mapreduce as it's own class.  Then write a wrapper module whose sole job is to combine those mapreduces by overriding `steps`.

Some simple boilerplate for this, taking advantage of the default `steps` function that we get for free in a single-step MRJob class:

```python
class FirstStep(MRJob):
  def mapper(self, key, value):
    pass
  def reducer(self, key, values):
    pass
  
class SecondStep(MRJob):
  def mapper(self, key, value):
    pass
  def reducer(self, key, values):
    pass
  
class SteppedJob(MRJob):
  """
  A two-step job that first runs FirstStep's MR and then SecondStep's MR
  """
  def steps(self):
    return FirstStep().steps() + SecondStep().steps()
```


### Note on Style
Here are some helpful articles on how mrjob works and how to pass parameters to your script:
  - [How mrjob is run](https://pythonhosted.org/mrjob/guides/concepts.html#how-your-program-is-run)
  - [Adding passthrough options](https://pythonhosted.org/mrjob/job.html#mrjob.job.MRJob.add_passthrough_option)
  - [An example of someone solving similar problems](http://arunxjacob.blogspot.com/2013/11/hadoop-streaming-with-mrjob.html)

See the notebook "Hadoop MapReduce with mrjob" in the datacourse for more details.

Finally, if you are find yourself processing a lot of special cases, you are probably doing it wrong.  For example, mapreduce jobs for `Top100WordsSimpleWikipediaPlain`, `Top100WordsSimpleWikipediaText`, and `Top100WordsSimpleWikipediaNoMetaData` are less than 150 lines of code (including generous blank lines and biolerplate code).

## Question 1: top100_words_simple_plain
Return a list of the top 100 words in an article text (in no particular order). You will need to write this as two map reduces:

1. The first job is similar to standard wordcount but with a few tweaks. The data provided for wikipedia is in `*.xml.bz2` format.  Mrjob will automatically decompress `bz2`.  We'll deal with the `xml` in the next question. For now, just treat it as text.  A few hints:
   - To split the words, use the regular expression "\w+".
   - Words are not case sensitive: i.e. "The" and "the" reference to the same word.  You can use `string.lower()` to get a single case-insenstive canonical version of the data.

2. The second job will take a collection of pairs `(word, count)` and filter for only the highest 100.  A few notes:
    - **Passing parameters:** To make the job more reusable make the job find the largest `n` words where `n` is a parameter obtained via [`get_jobconf_value`](https://pythonhosted.org/mrjob/utils-compat.html).
    - **Keeping track of the top n:** We have to keep track of at most the `n` most popular words.  As long as `n` is small, e.g. 100, we can keep track of the *running largest n* in memory wtih a priority-queue. We suggest taking a look at `heapq` ([details](https://docs.python.org/2/library/heapq.html)), part of the Python standard library for this.  It allows you to push elemnets into a list while keeping track of the highest priority element.
```python
h = []
heappush(h, (5, 'write code'))
heappush(h, (7, 'release product'))
heappush(h, (1, 'write spec'))
heappush(h, (3, 'create tests'))
heappop(h)  // returns (1, 'write spec')
```
   
       A naive implementation would cost $O(1)$ to insert but $O(n)$ to retrieve.  `heapq` uses a [self-balancing binary search tree](https://en.wikipedia.org/wiki/Self-balancing_binary_search_tree) to enable $O(\log(n))$ insertion and $O(1)$ removal. You may be asked about this data structure on an interview so it is good to get practice with it now.
    - **Working across nodes:** To obtain the largest `n`, we need to first obtain the largest n elements per chunk from the mapper, output them to the same key (reducer), and then collect the largest n elements of those in the reducer (**Question:** why does this gaurantee that we have found the largest n over the entire set?)
    - **Working within a node:** Given that we are using a priority queue, we will need to first initialize it, then `push` or `pushpop` each record to it, and finally output the top `n` after seeing each record.  For mappers, notice that these three phases correspond nicely to these three functions:
        - `mapper_init`
        - `mapper`
        - `mapper_final`

    There are similar functions in the reducer.  Also, while the run method to launch the mapreduce job is a classmethod:
        ```python
          if __name__ == '__main__':
            MRWordCount.run()
        ```
     actual instances of our mapreduce are instantiated on the map and reduce nodes.  More precisely, a separate mapper class is instantiated in each map node and a reducer class is instantiated in each reducer node.  This means that the three mapper functions can pass state through `self`, e.g. `self.heap`. Remember that to pass state between the map and reduce phase, you will have to use `yield` in the mapper and read each line in the reducer. (**Question:** Can you pass state between two mappers?)

**Checkpoint:**
- Total unique words: 1,584,646

In [73]:
l = []
with open('/home/vagrant/datacourse/MR/output.txt') as f:
    lines = f.read().replace('"', '').splitlines()
output_t = [tuple(x.split('\t')) for x in lines]
for i in range(200):
    l.append(tuple((output_t[i][0],int(output_t[i][1]))))

In [82]:
l[:100]


[('the', 1572530),
 ('quot', 1400092),
 ('gt', 1211881),
 ('lt', 1205636),
 ('id', 1142726),
 ('of', 972021),
 ('in', 658172),
 ('text', 604105),
 ('title', 539585),
 ('to', 488669),
 ('page', 439533),
 ('is', 406945),
 ('format', 386248),
 ('model', 381440),
 ('revision', 380458),
 ('category', 380445),
 ('ns', 378528),
 ('timestamp', 377859),
 ('contributor', 377213),
 ('sha1', 376839),
 ('ref', 370300),
 ('username', 361648),
 ('comment', 349660),
 ('parentid', 292767),
 ('on', 243819),
 ('by', 218299),
 ('http', 217151),
 ('for', 215453),
 ('was', 213271),
 ('it', 209568),
 ('x', 201333),
 ('wiki', 199371),
 ('space', 197034),
 ('xml', 189057),
 ('preserve', 188765),
 ('wikitext', 188559),
 ('name', 179580),
 ('d', 157977),
 ('that', 157381),
 ('www', 156201),
 ('s', 154554),
 ('font', 153498),
 ('user', 153378),
 ('from', 152592),
 ('with', 137737),
 ('style', 136494),
 ('he', 125011),
 ('this', 122896),
 ('com', 122511),
 ('color', 121251),
 ('talk', 120486),
 ('minor', 117012),


In [83]:
def top100_words_simple_plain():
    # return [("the", 1586419)] * 100
    return l[:100]
grader.score(question_name='mr__top100_words_simple_plain', func=top100_words_simple_plain)

Your score:  0.76


## Question 2: top100_words_simple_text
Notice that the words "page" and "text" make it into the top 100 words in the previous problem.  These are not common English words!  If you look at the xml formatting, you'll realize that these are xml tags.  You should parse the files so that tags like `<page></page>` should not be included in your total, nor should words outside of the tag `<text></text>`.

**Hints**:
1. Both `xml.etree.elementtree` from the Python stdlib or `lxml.etree` parse xml. `lxml` is significantly faster though and avoids some bugs.

2. In order to parse the text, we will have to accumulate a `<page></page>` worth of data and then split the resulting string into words.

3. Don't forget that the Wikipedia format can have multiple revisions but you only want the latest one.

4. What happens if a content from a page is split across two different mappers? How does this problem scale with data size?

**Checkpoint:**
- Total unique words: 867,871

In [ ]:
def top100_words_simple_text():
    return [("the", 1577579)] * 100

grader.score(question_name='mr__top100_words_simple_text', func=top100_words_simple_text)

## Question 3: top100_words_simple_no_metadata

Finally, notice that 'www' and 'http' make it into the list of top 100 words in the previous problem.  These are also not common English words either!  These are clearly from the url in hyperlinks.  Looking at the format of [Wikipedia links](http://en.wikipedia.org/wiki/Help:Wiki_markup#Links_and_URLs) and [citations](http://en.wikipedia.org/wiki/Help:Wiki_markup#References_and_citing_sources), you'll notice that they tend to appear within single and double brackets and curly braces.

**Hint**:
You can either write a simple parser to eliminate the urls within brackets, angle braces, and curly braces or you can use a package like the colorfully-named [mwparserfromhell](https://github.com/earwig/mwparserfromhell/), which has been provisioned on `mrjob` and supports the convenient helper function `strip_code()` (which is used by the reference solution).

**Checkpoint:**
- Total unique words: 618,410

In [ ]:
def top100_words_simple_no_metadata():
    return [("the", 1427342)] * 100

grader.score(question_name='mr__top100_words_simple_no_metadata', func=top100_words_simple_no_metadata)

## Question 4: wikipedia_entropy
The [Shannon entropy](https://en.wikipedia.org/wiki/Entropy_(information_theory) of a discrete random variable with probability mass function $p(x)$ is:

$$H(X) = - \sum p(x) \log_2 p(x)$$

You can think of the Shannon entropy as the number of bits needed to represent the random variable if it were optimally compressed.  It is also closely tied to the notion of entropy from physics.

You'll be estimating the Shannon entropy of different Simple English and Thai based off of their Wikipedias. Do this with n-grams of characters, by first calculating the entropy of a single n-gram and then dividing by n to get the per-character entropy. Use n-grams of size 1, 2, 3.  How should our per-character entropy estimates vary with n?  How should they vary by
the size of the corpus? How much data would we need to get reasonable entropy estimates for each n?

The data you need is available at:
    - https://s3.amazonaws.com/dataincubator-course/mrdata/simple/part-000\*
    - https://s3.amazonaws.com/dataincubator-course/mrdata/thai/part-000\*

*Question*: Why do we need to use map-reduce? There are >300 million characters in this dataset. How much memory would it take to store all `n`-grams as `n` increases?

Notes:
1. Characters are case sensitive.
1. Do not use the previous regex `\w+` to split --- depending on your system configuration, this may only match English characters, which would severely skew entropy estimates for Thai. Be careful about unicode.
1. Please treat all whitespace as the same character.  You can do this by
  `" ".join(text.split())`
1. For reference, the exact code we use to extract text is:

```
    wikicode = mwparserfromhell.parse(text)
    text = " ".join(" ".join(fragment.value.split())
                    for fragment in wikicode.filter_text())
```

A naive implementation of this job will take a very long time to run.  Instead, we will need to use a few optimizations:
1. See [this post](http://www.johndcook.com/blog/2013/08/17/calculating-entropy/) on how to calculate entropy efficiently.
2. It turns out that writing to disk is the most expensive part of a map-reduce.  [Zipf's law](https://en.wikipedia.org/wiki/Zipf's_law) tells us that only a handful (relatively-speaking) of n-grams make up most of our observations.  Can you do a map-side cache of these values to reduce the number of writes?
   
   Note that it may not be possible to match the reference solution by omitting n-grams.
3. Entropy is a function of the count distribution, i.e. it is independent of which ngrams correspond to which counts.  If we have N singleton ngrams, it's more efficient to (somehow) encode that as "N singleton ngrams" rather than as N key-value pairs:
```
    (word1, 1)
    (word2, 1)
    (word3, 1)
    ...
```
   Can you use a in-memory cache to solve this problem?  What fraction of ngrams only occur once?  How much of a speedup do you expect to get from this optimization?

In [ ]:
def wikipedia_entropy():
    return [
        ("simple1", 1.),
        ("simple2", 1.),
        ("simple3", 1.),
        ("thai1", 1.),
        ("thai2", 1.),
        ("thai3", 1.),
    ]

grader.score(question_name='mr__wikipedia_entropy', func=wikipedia_entropy)

## Question 5: link_stats_simple
Let's look at some summary statistics on the number of unique links on a page to other Wikipedia articles.  Return the number of articles (count), average number of links, standard deviation, and the 25%, median, and 75% quantiles.

1. Notice that the library `mwparserfromhell` supports the method `filter_wikilinks()`.
2. You will need to compute these statistics in a way that requires O(1) memory.  You should be able to compute the first few (i.e. non-quantile) statistics exactly by looking at the first few moments of a distribution. The quantile quantities can be accurately estimated by using reservoir sampling with a large reservoir.
3. If there are multiple links to the article have it only count for 1.  This keeps our results from becoming too skewed.
4. Don't forget that some (a surprisingly large number of) links have unicode! Make sure you treat them correctly.

In [ ]:
def link_stats_simple():
    return [
        ("count", 0.),
        ("mean", 0.),
        ("stdev", 0.),
        ("25%", 0.),
        ("median", 0.),
        ("75%", 0.),
    ]

grader.score(question_name='mr__link_stats_simple', func=link_stats_simple)

## Question 6: link_stats_english
The same thing but for all of English Wikipedia.  This is the real test of how well your algorithm scales!  The data is also located on [s3](s3://dataincubator-course/mrdata/english/).

**Note:**
Because of the size of the dataset, this job may take several hours to complete. It's advisable to run it overnight once you're reasonably sure it will work (due to testing the code on smaller inputs).

As a barometer, our reference solution takes around 5 hours to run.

In [ ]:
def link_stats_english():
    return [
        ("count", 0.),
        ("mean", 0.),
        ("stdev", 0.),
        ("25%", 0.),
        ("median", 0.),
        ("75%", 0.),
    ]

grader.score(question_name='mr__link_stats_english', func=link_stats_english)

## Question 7: double_link_stats_simple
Instead of analyzing single links, let's look at double links.  That is, pages `A` and `C` that are connected through many pages `B` where there is a link `A -> B -> C` or `C -> B -> A`. Find the list of all pairs `(A, C)` (you can use alphabetical ordering to break symmetry) that have the 100 "most" connections (see below for the definition of "most").  This should give us a notion that the articles `A` and `C` refer to tightly related concepts.

1. This can be thought of as a Matrix Multiplication problem.  If the adjacency matrix is denoted $M$ (where $M_{ij}$ represents the link between $i$ an $j$), we are looking for the highest 100 elements of the matrix $M M$. Note that this doesn't mean constructing matrices is the most efficent way to solve the problem.

2. Notice that a lot of Category pages (denoted "Category:.*") have a high link count and will rank very highly according to this metric.  Wikipedia also has `Talk:` pages, `Help:` pages, and static resource `Files:`.  All such "non-content" pages (and there might be more than just this) and links to them should be first filtered out in this analysis.

3. Some pages have more links than others.  If we just counted the number of double links between pages, we will end up seeing a list of articles with many links, rather than concepts that are tightly connected.

   1. One strategy is to weight each link as $\frac{1}{n}$ where $n$ is the number links on the page.  This way, an article has to spread it's "influence" over all $n$ of its links.  However, this can throw off the results if $n$ is small.

   2. Instead, try using [Bayesian Smoothing](https://en.wikipedia.org/wiki/Additive_smoothing) by weighting each link as $\frac{1}{n+10}$ where 10 sets the "scale" in terms of number of links above which a page becomes "significant".  The number 10 was somewhat arbitrarily chosen but seems to give reasonably relevant results.

   3. This means that our "count" for a pair A,C will be the products of the two link weights between them, summed up over all their shared connections.

4. Again, if there are multiple links from a page to another, have it only count for 1.  This keeps our results from becoming skewed by a single page that references the same page multiple times.

5. You'll need to filter out the cases where A -> B -> A, but just note that the reference solution does this after calculating the link weights.

6. The links and page titles are encoded into utf-8.

Don't be afraid if these answers are not particularly insightful.  Simple Wikipedia is not as rich as English Wikipedia.  However, you should notice that the articles are closely related conceptually.


In [ ]:
def double_link_stats():
    return [(("idaho", "list of cities in idaho"), 0.055922694796753325)] * 100

grader.score(question_name='mr__double_link', func=double_link_stats)

*Copyright &copy; 2016 The Data Incubator.  All rights reserved.*